In [82]:
library(tidyverse)
library(plotly)
library(shiny)

codes1 = read.csv("./data/2022/DC-2022/NIBRS_OFFENSE_TYPE.csv")
codes2 = read.csv("./data/2020/DC-2020/NIBRS_OFFENSE_TYPE.csv")

# 22,21  /  20,19,18

# Simple Assault  13B / 51
# Destruction/Damage/Vandalism of Property 290 / 5
# Theft From Motor Vehicle 23F / 14
# Motor Vehicle Theft 240 / 21
# Intimidation 13C / 44
# identity theft 26F / 63
# robbery 120 / 40
# Drug/Narcotic Violations 35A / 16
# All Other Larceny 23H / 45

In [83]:
extract_offense = function(base_dir, year){
    state_dfs = list() 
    for(state in list.files(paste0(base_dir, year))){
        file_path = paste0(base_dir,year,"/",state,"/NIBRS_OFFENSE.csv")
        df = read.csv(file_path)
        df_name = paste0("offense_", substring(state,1,2),"_",year)
        assign(df_name, df)
        state_dfs[[substring(state,1,2)]] = df
    }
    return(state_dfs)
}

count_offense = function(offense_codes, offense_df){
  n_offenses = sapply(offense_df, function(df){
    lengths = sapply(offense_codes, function(offense_code){
      if(offense_code == "ALL"){
          length(unique(df$offense_id))
      } else {
          sum(df$offense_code == offense_code)
      }
    })
    names(lengths) = offense_codes
    lengths
  })
  n_offenses = as.data.frame(t(n_offenses))
  n_offenses$state_abb = row.names(n_offenses)
  row.names(n_offenses) = NULL
  return(n_offenses)
}

count_offense2 = function(offense_codes, offense_df){
  n_offenses = sapply(offense_df, function(df){
    lengths = sapply(offense_codes, function(offense_code){
      if(offense_code == "ALL"){
          length(unique(df$OFFENSE_ID))
      } else {
          sum(df$OFFENSE_TYPE_ID == offense_code)
      }
    })
    names(lengths) = offense_codes
    lengths
  })
  n_offenses = as.data.frame(t(n_offenses))
  n_offenses$state_abb = row.names(n_offenses)
  row.names(n_offenses) = NULL
  return(n_offenses)
}

get_offense_code = function(codes, offense_name){
  offense_code = codes[codes$offense_name == offense_name, "offense_code"]
  if(offense_name == "ALL"){
    return(offense_name)
  } else {
    return(offense_code)
  }
}

get_offense_code2 = function(codes, offense_name){
  offense_code = as.character(codes[codes$OFFENSE_NAME == offense_name, "OFFENSE_TYPE_ID"])
  if(offense_name == "ALL"){
    return(offense_name)
  } else {
    return(offense_code)
  }
}

base_dir = "./data/"
offense_codes1 = c("ALL", "13B", "290", "23F", "240", "13C", "26F", "120", "35A", "23H")
offense_codes2 = c("ALL", "51", "5", "14", "21", "44", "63", "40", "16", "45")

offense_22 = extract_offense(base_dir, year=2022)
offense_21 = extract_offense(base_dir, year=2021)
offense_20 = extract_offense(base_dir, year=2020)
offense_19 = extract_offense(base_dir, year=2019)
offense_18 = extract_offense(base_dir, year=2018)

n_offense_22 = count_offense(offense_codes1, offense_22)
n_offense_21 = count_offense(offense_codes1, offense_21)
n_offense_20 = count_offense2(offense_codes2, offense_20)
n_offense_19 = count_offense2(offense_codes2, offense_19)
n_offense_18 = count_offense2(offense_codes2, offense_18)

n_offense = list("2022" = n_offense_22, "2021" = n_offense_21, "2020" = n_offense_20, "2019" = n_offense_19, "2018" = n_offense_18)

In [85]:
types = c("ALL", "Simple Assault", "Motor Vehicle Theft", "Intimidation", "Identity Theft", "Robbery")

g = list(
  scope = 'usa',
  projection = list(type = 'albers usa'),
  showlakes = TRUE,
  lakecolor = toRGB('white')
)

ui = fluidPage(
  titlePanel("Crimes in United States by offense type"),
  sidebarLayout(
    sidebarPanel(
      selectInput("yearInput", "Year:", choices = c("2018","2019","2020","2021","2022")),
      selectInput("offenseInput", "Offense Type:", choices = types)
    ),
    mainPanel(
      plotlyOutput("crimeMap")
    )
  )
)

server = function(input, output){
  output$crimeMap = renderPlotly({
    df = n_offense[[input$yearInput]]
    if(input$yearInput %in% c("2022", "2021")){
      offense_code = get_offense_code(codes1, input$offenseInput)
    } else if (input$yearInput %in% c("2020", "2019", "2018")){
      offense_code = get_offense_code2(codes2, input$offenseInput)
    }
    map = plot_geo(locationmode = 'USA-states')
    map = map %>% add_trace(
      z = df[[offense_code]], locations = df$state_abb,
      color = df[[offense_code]], colors = "Purples")
    map = map %>% layout(
      title = "Offenses",
      geo = g)
    map
  })
}

shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:6349



https://github.com/coatless-quarto/r-shinylive-demo